## 딥러닝 실습 과제 3주차 - CNN을 활용한 이미지 데이터 증강 실험

다음  네 가지 활동을 해봅시다.

01. **데이터 전처리 및 로딩**: "기존" vs "기존 + 증강"
02. **모델 설계**: CNN 기반 분류 모델 재사용 및 수정
03. **손실 함수 정의**: 각 칸에 대해 3-클래스 분류
04. **증강 적용 유무에 따른 성능 비교**

TTTDataset.zip을 불러와 문제에서 요하는 코드를 구현하세요.

💡 **데이터 구조**  
- **`image_black`** : 이미지 데이터  
- **`labels`** : 타겟 데이터  

In [131]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [132]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import json
import glob
import os

In [133]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## 00. 클래스
정의한 클래스를 이용해 실행해 주세요.

In [134]:
class TTTDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        """
        틱택토 데이터셋을 PyTorch Dataset 형태로 변환.
        :param image_paths: 이미지 파일 경로 리스트
        :param label_paths: 레이블 JSON 파일 경로 리스트
        :param transform: 이미지 전처리 변환
        """
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
        self.data = self._load_data()


    def _load_data(self):
        """ 이미지 & 레이블 로드 """
        data = []
        for img_path, lbl_path in zip(self.image_paths, self.label_paths):
            # 이미지를 흑백(Grayscale)로 변환
            image = Image.open(img_path).convert("L")  # "RGB" 대신 "L" 사용

            # JSON 레이블 로드
            with open(lbl_path, 'r') as f:
                labels = json.load(f)

            # 레이블을 숫자로 변환 (O=1, X=-1, blank=0)
            label_tensor = torch.tensor(
                [1 if v == "O" else -1 if v == "X" else 0 for v in labels.values()],
                dtype=torch.float32
            )
            data.append((image, label_tensor))

        return data


    def __len__(self):
        """ 데이터셋 크기 반환 """
        return len(self.data)


    def __getitem__(self, idx):
        """ 데이터셋에서 idx 번째 샘플(이미지 & 레이블)을 가져오는 역할 """
        image, label = self.data[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [135]:
# 압축 파일 풀기
zip_path = "/content/drive/MyDrive/TTTDataset.zip"
extract_path = "/content/TTTDataset"

if not os.path.exists(extract_path):
    !unzip -q "{zip_path}" -d "/content/"
    print(" 압축 해제 완료!")
else:
    print(" 이미 압축이 풀려 있으므로 건너뜁니다.")

 이미 압축이 풀려 있으므로 건너뜁니다.


## 01. 데이터 전처리 및 로딩: "기존" vs "기존 + 증강"

💡 실험을 위한 **두 개의 DataLoader**를 구성하세요 (1주차 과제 참고)
- 기존 데이터만 사용하는 DataLoader
- 기존 데이터 + 실시간 transform 증강을 적용한 DataLoader


In [136]:
from torchvision import transforms

In [137]:
# 이미지 및 레이블 디렉토리 경로 설정
image_dir = "/content/TTTDataset/image_black"
label_dir = "/content/TTTDataset/labels"

# 이미지와 라벨 파일 자동 수집 (확장자가 .jpg 또는 .JPG인 점 확인)
image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")) +
                     glob.glob(os.path.join(image_dir, "*.JPG")))

label_paths = sorted(glob.glob(os.path.join(label_dir, "*.json")))

각자 이미지 전처리 실행!

In [138]:
# 기본 transform
base_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(), # 픽셀값을 텐서로 변환 (0~1로 자동 스케일링됨)
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# 증강 transform
aug_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(15), # 주어진 각도에 대해 랜덤하게 회전
    transforms.ColorJitter(brightness=0.3, contrast=0.3), # 컬러 관련 속성 변경
    transforms.GaussianBlur(3), # 각 픽셀 주변의 값을 평균 내어 부드럽게 처리 / 노이즈 제거
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

각자 Dataset 및 Dataloader 정의!

In [139]:
# 기본 Dataset 생성
base_dataset = TTTDataset(image_paths, label_paths, transform=base_transform)

# 분할 비율 설정 (예: 70% train, 15% val, 15% test)
total_size = len(base_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# 기본 Dataset 분할
train_base_dataset, val_base_dataset, test_base_dataset = random_split(base_dataset, [train_size, val_size, test_size])

# 기본 DataLoader 생성
train_loader = DataLoader(train_base_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_base_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_base_dataset, batch_size=32, shuffle=True)

💡 **증강**은 "실시간 변형"을 뜻한다.
- 동일한 이미지 경로/레이블 경로를 참조하기 때문에 개수가 완전히 동일하다! 따라서 **데이터 개수**가 늘어나는게 아니라 같은 개수의 데이터를 매번 **다르게 변형해서 학습에 사용**한다는 점~
-  **`Subset(aug_dataset, train_base_dataset.indices)`**을 쓰는 이유는 **같은 데이터 분할 기준을 유지하면서 transform만 다르게 하기 위함**이다!
- aug_dataset도 **`random_split`**으로 나눠버리면 **기존 학습용 데이터와 전혀 다른 데이터로 학습**하게 돼서 **transform의 효과를 비교하는 실험이 되지 않는다!**

In [140]:
# 증강 Dataset 생성
aug_dataset = TTTDataset(image_paths, label_paths, transform=aug_transform)

# 증강 Dataset 분할 (이때 random_split을 사용하지 않고, 동일한 분할 인덱스를 활용한다.)
train_aug_dataset = torch.utils.data.Subset(aug_dataset, train_base_dataset.indices)

# 증강 DataLoader 생성 (이때 val_loader 와 test_loader는 기본 Dataset의 것을 차용하기 때문에 별도로 생성하지 않는다.)
train_aug_loader = DataLoader(train_aug_dataset, batch_size=32, shuffle=True)
'''
val_loader = DataLoader(val_base_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_base_dataset, batch_size=32, shuffle=True)
'''

'\nval_loader = DataLoader(val_base_dataset, batch_size=32, shuffle=True)\ntest_loader = DataLoader(test_base_dataset, batch_size=32, shuffle=True)\n'

## 02. 모델 설계: CNN 기반 분류 모델 재사용 및 수정
- 1채널 이미지 입력, 9 × 3-클래스 출력 구조 유지 (2주차 과제 참고)
- Dropout/Hidden Layer 등 수정 가능


In [141]:
import torch.nn as nn

class TicTacToeCNN(nn.Module):
    def __init__(self):
        super(TicTacToeCNN, self).__init__()
        # 첫번째 층: 1채널 -> 32채널, BatchNorm 추가
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # (B, 32, 64, 64)
        )

        # 두번째 층: 32채널 -> 64채널, BatchNorm 추가
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # (B, 64, 32, 32)
        )

        # 세번째 층: 64채널 -> 128채널, BatchNorm 추가
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))           # (B, 128, 1, 1)
        )

        # Fully-connected Layer: 확장된 구조로 수정
        self.fc_layer = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),                       # Dropout 비율 약간 증가
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 9 * 3)                   # 최종 출력: 9칸 x 3 클래스
        )

    def forward(self, x):
        x = self.layer1(x)              # (B, 32, 64, 64)
        x = self.layer2(x)              # (B, 64, 32, 32)
        x = self.layer3(x)              # (B, 128, 1, 1)
        x = x.view(x.size(0), -1)       # (B, 128)
        x = self.fc_layer(x)            # (B, 27)
        return x.view(-1, 9, 3)         # (B, 9, 3)

## 03. 손실 함수 정의: 각 칸에 대해 3-클래스 분류
- 다중 클래스 분류이므로 CrossEntropyLoss 사용 가능
- 9개 칸을 각각 분류하는 방식으로 모델 구성
- Adam 옵티마이저 사용 권장

In [142]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 모델 객체 선언
model = TicTacToeCNN().to(device)

# 손실 함수 정의 (다중 클래스 분류)
criterion = nn.CrossEntropyLoss()
# 최적화 알고리즘 정의
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## 04. 증강 적용 유무에 따른 성능 비교
- 두 개의 DataLoader로 두 번 실험
- 같은 모델 구조로 학습 → 성능 차이 비교 (정확도/학습 안정성/과적합 여부 등)

In [143]:
# 전처리 함수 정의
def preprocess_batch(images, labels):
    images = images.to(device)
    labels = labels.to(device)
    # CrossEntropy는 정수 클래스 (0, 1, 2)를 기대함
    target = (labels + 1).to(torch.int64)   # -1 → 0, 0 → 1, 1 → 2로 정수형 변환
    return images, target

In [144]:
# 학습 함수 정의
def train_model(model, loader, optimizer, criterion, val_loader, epochs=30):
    model.train()
    for epoch in range(epochs):
      # 누적 손실 초기화
      total_loss = 0

      for images, labels in loader:
          images, target = preprocess_batch(images, labels)

          optimizer.zero_grad() # 기울기 초기화
          outputs = model(images)  # (B, 9, 3)
          loss = criterion(outputs.view(-1, 3), target.view(-1)) # 손실 계산
          loss.backward() # 역전파
          optimizer.step() # 가중치 업데이트
          total_loss += loss.item() # 손실 누적
      val_acc = evaluate(model, val_loader)
      print(f"[Epoch {epoch+1:02d}] Loss: {total_loss:.4f} | Val Accuracy: {val_acc:.2%}")
    return model

In [145]:
# 평가 함수 정의
def evaluate(model, loader):  # train_loader, val_loader, test_loader 중 하나를 넘길 수 있음
    model.eval()
    # 예측이 맞은 칸 수
    correct = 0
    # 전체 칸 수
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, target = preprocess_batch(images, labels)
            # 모델에 이미지 삽입하여 예측 수행
            outputs = model(images)  # (B, 9, 3)
            # 각 칸마다 3-클래스 중 가장 높은 점수를 선택
            preds = outputs.argmax(dim=2)  # (B, 9)
            correct += (preds == target).sum().item()
            total += target.numel()

    return correct / total

**`get_new_model()`**은 두 실험의 조건을 공정하게 분리하기 위해 꼭 필요한 함수이다.
- 우리가 비교하려는 건 **기존 데이터로 학습한 모델 vs 증강 데이터로 학습한 모델** 이기 때문에, 매 실험마다 **"새 모델"**로 시작해야 한다!
- 모델을 새로 만들지 않는다면 두 번째 학습이 첫 번째 결과 위에 덮어씌워져서 실험 결과가 왜곡된다.

In [146]:
# 모델 정의
def get_new_model():
    model = TicTacToeCNN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    return model, optimizer, criterion

- **기존 데이터 vs 증강 데이터**에 대해 각각 학습하고,
- **동일한 검증셋**에서 최종 성능을 비교하는 코드

In [147]:
# 실험 1: 증강 없이 기본 이미지 학습
print("실험 1: 증강 없이 기본 이미지 학습")
model1, optimizer1, criterion1 = get_new_model() # 모델, 옵티마이저, 손실 함수 초기화
model1 = train_model(model1, train_loader, optimizer1, criterion1, val_loader)

실험 1: 증강 없이 기본 이미지 학습
[Epoch 01] Loss: 10.8884 | Val Accuracy: 35.99%
[Epoch 02] Loss: 10.7427 | Val Accuracy: 40.30%
[Epoch 03] Loss: 10.5583 | Val Accuracy: 41.29%
[Epoch 04] Loss: 10.2593 | Val Accuracy: 46.43%
[Epoch 05] Loss: 9.8438 | Val Accuracy: 49.92%
[Epoch 06] Loss: 9.4804 | Val Accuracy: 49.09%
[Epoch 07] Loss: 9.2561 | Val Accuracy: 52.57%
[Epoch 08] Loss: 9.1575 | Val Accuracy: 48.26%
[Epoch 09] Loss: 9.0450 | Val Accuracy: 50.75%
[Epoch 10] Loss: 8.9832 | Val Accuracy: 50.08%
[Epoch 11] Loss: 8.9060 | Val Accuracy: 49.75%
[Epoch 12] Loss: 8.9235 | Val Accuracy: 51.91%
[Epoch 13] Loss: 8.8746 | Val Accuracy: 49.59%
[Epoch 14] Loss: 8.7892 | Val Accuracy: 51.74%
[Epoch 15] Loss: 8.7273 | Val Accuracy: 51.91%
[Epoch 16] Loss: 8.6471 | Val Accuracy: 50.91%
[Epoch 17] Loss: 8.5947 | Val Accuracy: 50.41%
[Epoch 18] Loss: 8.6138 | Val Accuracy: 51.58%
[Epoch 19] Loss: 8.6093 | Val Accuracy: 51.08%
[Epoch 20] Loss: 8.6916 | Val Accuracy: 53.57%
[Epoch 21] Loss: 8.5716 | Val Accu

In [148]:
# 실험 2: 증강 이미지 학습
print("실험 2: 증강 이미지 학습")
model2, optimizer2, criterion2 = get_new_model()
model2 = train_model(model2, train_aug_loader, optimizer2, criterion2, val_loader)

실험 2: 증강 이미지 학습
[Epoch 01] Loss: 10.8779 | Val Accuracy: 37.15%
[Epoch 02] Loss: 10.7709 | Val Accuracy: 40.96%
[Epoch 03] Loss: 10.4949 | Val Accuracy: 43.12%
[Epoch 04] Loss: 10.1200 | Val Accuracy: 47.26%
[Epoch 05] Loss: 9.6990 | Val Accuracy: 52.24%
[Epoch 06] Loss: 9.5037 | Val Accuracy: 51.74%
[Epoch 07] Loss: 9.2761 | Val Accuracy: 52.07%
[Epoch 08] Loss: 9.2268 | Val Accuracy: 51.91%
[Epoch 09] Loss: 9.1988 | Val Accuracy: 51.24%
[Epoch 10] Loss: 9.1362 | Val Accuracy: 52.57%
[Epoch 11] Loss: 9.0205 | Val Accuracy: 51.24%
[Epoch 12] Loss: 9.0104 | Val Accuracy: 52.90%
[Epoch 13] Loss: 8.9514 | Val Accuracy: 51.24%
[Epoch 14] Loss: 8.9055 | Val Accuracy: 52.74%
[Epoch 15] Loss: 8.8935 | Val Accuracy: 52.07%
[Epoch 16] Loss: 8.8462 | Val Accuracy: 50.75%
[Epoch 17] Loss: 8.8554 | Val Accuracy: 52.07%
[Epoch 18] Loss: 8.8442 | Val Accuracy: 49.42%
[Epoch 19] Loss: 8.8196 | Val Accuracy: 51.41%
[Epoch 20] Loss: 8.8124 | Val Accuracy: 53.90%
[Epoch 21] Loss: 8.7590 | Val Accuracy: 

In [149]:
# 최종 비교: 두 모델을 같은 기준으로 평가
final_acc1 = evaluate(model1, val_loader)
final_acc2 = evaluate(model2, val_loader)
print(f"Final Validation Accuracy (Base): {final_acc1:.2%}")
print(f"Final Validation Accuracy (Augmented): {final_acc2:.2%}")

Final Validation Accuracy (Base): 52.24%
Final Validation Accuracy (Augmented): 51.74%
